# This is the Big Fancy ELO Calculator Thing
Using an imported sheet, this updates the main database sheet with a new game.
This uses a modified version of ELO that I developed a while ago.
It might not be 100% mathematically sound. Therefore, this is still a prototype! 
This WILL NOT WORK for any rolelist that has 3 or more competing factions, ie Powered Mafia Vendetta.
It also WILL NOT WORK for Neutral roles that convert players, since that creates a 'team' that isn't accounted for.
I don't know who I'm sharing this with, but if you, code reader, wish to contact me, I am:
@Actionguy777#2115 on Discord
@Actionguy7771 on Twitter

I have no other social media :)

In [157]:
#imports, not sure if pip is actually used anywhere

import numpy as np
import pandas as pd
import pip

#importing the sheets. newgame_sheet can be changed.
#there's no filepath here- this program is in the same folder as the sheets used.

main_sheet = "FullSheet.xlsx"
main_dataframe = pd.read_excel(main_sheet, index_col = 0)
#display(main_dataframe)

newgame_sheet = "ELO_TEST_SHEET.xlsx"
newgame_dataframe = pd.read_excel(newgame_sheet)
#display(newgame_dataframe)

In [158]:
#this function checks to make sure that the players in the game actually exist.
#If the player doesn't exist in the main database, they're added with 1000s for each ELO

#!!! BE CAREFUL! ANY DIFFERENT NAME WILL CREATE A NEW PLAYER ROW, EVEN IF IT'S A TYPO!
#!!! TO AVOID ISSUES, CHECK TO BE SURE THERE ARE NO TYPOS ON THE NEWGAME_SHEET

#i = 1 for testing purposes
for ind in newgame_dataframe.index:
    name = (newgame_dataframe['PLAYER'][ind])
    #print(str(i) + " is i's value for " + name) for testing purposes
    if name not in main_dataframe.values:
        #print(name + " is not in main!") for testing purposes
        main_dataframe.loc[len(main_dataframe.index)] = [name,1000,0,1000,0,1000,0,1000]
        #print("HERE!") for testing purposes

In [159]:
towndict = {}
mafdict = {}
neutdict = {}

def AddPlayerToDict(player_name, dictionary, check_col):
    select_indices = list(np.where(main_dataframe["NAME"] == player_name)[0])
    player_ELO = (main_dataframe.loc[select_indices[0], check_col])
    dictionary[player_name] = player_ELO

#Adds players to either a town or mafia dictionary.
#These dictionaries are used for several purposes, ultimately transforming into ELO changes.

for ind in newgame_dataframe.index:
    if (newgame_dataframe['FACTION'][ind] == "T"):
        player_name = newgame_dataframe['PLAYER'][ind]
        print(player_name + " is town!")
        AddPlayerToDict(player_name, towndict, 'TOWN_ELO')
    elif (newgame_dataframe['FACTION'][ind] == "M"):
        player_name = newgame_dataframe['PLAYER'][ind]
        print(player_name + " is mafia!")
        AddPlayerToDict(player_name, mafdict, 'MAFIA_ELO')
    elif (newgame_dataframe['FACTION'][ind] == "N"):
        player_name = newgame_dataframe['PLAYER'][ind]
        print(player_name + " is neutral!")
        AddPlayerToDict(player_name, neutdict, 'NEUTRAL_ELO')
        
#print(towndict)
#print(mafdict)
#print(neutdict)

#Getting averages for Town and Mafia, as well as average for all players.
#These are used both for creating a "Game ELO" for each player,
#And for comparing a player's Game ELO to the opponent team.
    
townTotal = sum(towndict.values())
mafiaTotal = sum(mafdict.values())
overallTotal = townTotal + mafiaTotal + sum(neutdict.values())

townCount = len(towndict)
mafiaCount = len(mafdict)
overallCount = len(neutdict) + townCount + mafiaCount
    
townAverage = townTotal/townCount
mafiaAverage = mafiaTotal/mafiaCount
totalAverage = overallTotal/overallCount

John is town!
Jack is town!
Jill is town!
June is town!
Jed is mafia!


In [160]:
#Averaging each player's category ELO with their team average for that game.
#I refer to this as "Game ELO", as it depends on a player's teammates' ELOs.

for key in towndict.keys():
    value = towndict[key]
    newvalue = (value+townAverage)/2 #this is used to calculate win chances
    towndict[key] = newvalue
for key in mafdict.keys():
    value = mafdict[key]
    newvalue = (value+mafiaAverage)/2 #this is used to calculate win chances
    mafdict[key] = newvalue
    
#Not necessary for neutral roles, since they are on a solo team.
#print(towndict)
#print(mafdict)
    
#Calculating each player's 'ELO score', to be compared to opposing team's average
townAverELO = 10**(townAverage/400)
mafAverELO = 10**(mafiaAverage/400)


#Calculating win chances using ELO scores.
def CalcWinChance(dictionary, opposingAverage):
    for key in dictionary:
        ELOVal = 10**(dictionary[key]/400)
        winChance = (ELOVal)/(ELOVal+opposingAverage)
        dictionary[key] = winChance
        
CalcWinChance(towndict, mafAverELO)
CalcWinChance(mafdict, townAverELO)

#Neutral players cannot use this function because they are different
#They compare their own ELO score to the combined ELO score of ALL OTHER PLAYERS (including other neutrals)
#!!! WARNING! This is NOT 100% accurate to how this should be calculated.
#Because neutral players are not necessarily 'against' everyone, it is dubious on who should be counted as an opponent
#This is a specific tangent that we would need to know individual roles for, which is outside the scope of this program.
for key in neutdict.keys():
    opposingAverage = (overallTotal - neutdict[key])/(overallCount - 1)
    opposingAverage = 10**(opposingAverage/400)
    ELOVal = 10**(neutdict[key]/400)
    winChance = (ELOVal)/(ELOVal+opposingAverage)
    neutdict[key] = winChance
    
print(towndict)
print(mafdict)
print(neutdict)

{'John': 0.5, 'Jack': 0.5, 'Jill': 0.5, 'June': 0.5}
{'Jed': 0.5}
{}


In [161]:
#Now we have the win chance of each player.
#The next step is to check each index of the game dataframe for a win or loss,
#and translate that into an ELO change for each dictionary key.
def ConvertWinChanceToELOChange(dictionary):
    for key in dictionary:
        select_indices = list(np.where(newgame_dataframe["PLAYER"] == key)[0])
        winbool = (newgame_dataframe.loc[select_indices[0], 'VICTORY'])
        contribution = (newgame_dataframe.loc[select_indices[0], 'CONTRIBUTION'])
        ELOChange = (-2*winbool+1) * (-1*dictionary[key]) * 15 * contribution #-2*winbool+1 is -1 if true, 1 if false.
        dictionary[key] = ELOChange

ConvertWinChanceToELOChange(towndict)
ConvertWinChanceToELOChange(mafdict)
ConvertWinChanceToELOChange(neutdict)
        
print(towndict)
print(mafdict)

{'John': 7.5, 'Jack': 7.5, 'Jill': 7.5, 'June': 1.875}
{'Jed': -7.5}


In [162]:
#This is where we add the changes to category ELO to the main dataframe!

def AddELOChangesToMainFrame(dictionary, ELO_col, GAMES_col): #dictionary, string, string
    for key in dictionary:
        select_indices = list(np.where(main_dataframe["NAME"] == key)[0])
        pre_ELO = (main_dataframe.loc[select_indices[0], ELO_col])
        new_ELO = pre_ELO + dictionary[key]
        main_dataframe.loc[select_indices[0], ELO_col] = round(new_ELO, 2)
        main_dataframe.loc[select_indices[0], GAMES_col] += 1

AddELOChangesToMainFrame(towndict, 'TOWN_ELO', 'TOWN_GAMES')
AddELOChangesToMainFrame(mafdict, "MAFIA_ELO", "MAFIA_GAMES")
AddELOChangesToMainFrame(neutdict, "NEUTRAL_ELO", 'NEUTRAL_GAMES')

In [163]:
#Next: Change the AVERAGE for each column in the AVERAGE row.
main_dataframe.loc[0] = ["AVERAGE",None,None,None,None,None,None,None]
#Change all the values to nans so we can calculate a column's average.
town_av = main_dataframe["TOWN_ELO"].mean()
maf_av = main_dataframe["MAFIA_ELO"].mean()
neut_av = main_dataframe["NEUTRAL_ELO"].mean()
main_dataframe.loc[0] = ["AVERAGE",town_av,None,maf_av,None,neut_av,None,None]

In [164]:
#This part calculates each player's "Full" ELO value.
#In other words, it aggregates a player's Town, Mafia, and Neutral ELO by
#using a weighted average.
#If you want the full explanation on how this is calculated, just ask and I'll bore you with it :)

def CalculateELO(t_ELO, t_games, m_ELO, m_games, n_ELO, n_games):
    multiplier = 1000
    weighted_av = (((t_ELO/town_av)*t_games)+((m_ELO/maf_av)*m_games)+((n_ELO/neut_av)*n_games))/(t_games+m_games+n_games)
    #print(weighted_av)
    return ((weighted_av)*multiplier)

for ind in main_dataframe.index:
    if ind == 0:
        print('skip')
    else:
        t_ELO = (main_dataframe['TOWN_ELO'][ind])
        t_games = (main_dataframe['TOWN_GAMES'][ind])
        m_ELO = (main_dataframe['MAFIA_ELO'][ind])
        m_games = (main_dataframe['MAFIA_GAMES'][ind])
        n_ELO = (main_dataframe['NEUTRAL_ELO'][ind])
        n_games = (main_dataframe['NEUTRAL_GAMES'][ind])
        new_ELO = CalculateELO(t_ELO, t_games, m_ELO, m_games, n_ELO, n_games)
        print(new_ELO)
        main_dataframe.loc[ind, 'FULL_ELO'] = round(new_ELO, 2)

skip
1003.422758856513
1003.422758856513
1003.422758856513
997.825502375348
993.7421777221526
999.0682249943753


In [40]:
#Finally, this exports the main_dataframe back to the main sheet!
#!!! WARNING! THIS IS IRREVERSIBLE! You might want to check what the main_dataframe looks like first just in case.
main_dataframe.to_excel("FullSheet.xlsx")

In [26]:
#newgame_dataframe

,PLAYER,FACTION,VICTORY,CONTRIBUTION
0,John,T,1,1.00
1,Jack,T,1,1.00
2,Jill,T,1,1.00
3,June,T,1,0.25
4,Jed,M,0,1.00


In [165]:
main_dataframe

,NAME,TOWN_ELO,TOWN_GAMES,MAFIA_ELO,MAFIA_GAMES,NEUTRAL_ELO,NEUTRAL_GAMES,FULL_ELO
0,AVERAGE,1004.063333,NaN,998.75,NaN,1000.0,NaN,NaN
1,John,1007.500000,1.0,1000.00,0.0,1000.0,0.0,1003.42
2,Jack,1007.500000,1.0,1000.00,0.0,1000.0,0.0,1003.42
3,Jill,1007.500000,1.0,1000.00,0.0,1000.0,0.0,1003.42
4,June,1001.880000,1.0,1000.00,0.0,1000.0,0.0,997.83
5,Jed,1000.000000,0.0,992.50,1.0,1000.0,0.0,993.74
6,Action,1000.000000,1.0,1000.00,1.0,1000.0,1.0,999.07
